Notebook disponível de forma interativa no [Binder](https://mybinder.org/v2/gh/QApedia/QApedia/master?filepath=examples).

In [1]:
# Carregando os módulos io, generator e utils do QApedia
from QApedia import io
from QApedia import generator
from QApedia import utils

# bibliotecas auxiliares
import pandas as pd
import csv

In [2]:
# Definir largura das colunas do dataframe para melhor visualização
pd.set_option('max_colwidth', 800)

# Definindo um template para geração de pares questão-sparql

Com o objetivo de gerar um dataset com mais amostras, pode-se realizar a definição de um conjunto menor de templates representando o formato das questões e suas respectivas consultas SPARQL. Um template de geração possui quatro campos:

1. Pergunta em linguagem natural.
    * Deve possuir lacunas de forma a manter a pergunta mais generalista possível. 
        * Por exemplo: **``<A>`` escreveu ``<B>``?**, gera:
            1. ``J. K. Rowling`` escreveu ``Harry Potter?``
            2. ``J. R. R. Tolkien`` escreveu ``Hobbit?``
2. Consulta SPARQL que deve responder a pergunta em linguagem natural.
    * Deve conter as mesmas lacunas presentes na pergunta.
    * A pseudo-query abaixo pode ser definida para a pergunta presente em 1.
        1. ASK where {``<B>`` escrito_por ``<A>``}
3. A SPARQL geradora deve ser capaz de recuperar as variáveis que possam preencher as lacunas
    * pseudo-query que retorna todos os ?a que foram escritos por ?b.
    ```
      select ?a ?b where {?b escrito_por ?a}
    ```
    * ?a e ?b são equivalentes as lacunas ``<A>`` e ``<B>``.

4. As variáveis da SPARQL geradora são no formato ?a ?b .. ?y ?z e correspondem as lacunas $<$A$>$ $<$B$>$ ... $<$Y$>$ $<$Z$>$ presentes na pergunta e na consulta.

Dessa forma, o template que constrói o conjunto de perguntas que verifica se "A é o autor de B?" pode ser definido assim:

```
question: <A> escreveu <B>?
query:  ASK where {<B> escrito_por <A>}
generator_query: select ?a ?b where {?b escrito_por ?a}
variables: [a, b]
```

A SPARQL geradora sempre é do tipo select, já que esta visa recuperar informações que serão utilizadas para preencher os campos das lacunas.

## QApedia - Formato do template

No QApedia o formato do template é um dicionário, no exemplo anterior, no python ficaria assim:

```python
template = {"question": "<A> escreveu <B>?",
            "query": "ASK where {<B> escrito_por <A>}",
            "generator_query": "select ?a ?b where {?b escrito_por ?a}",
            "variables": ["a", "b"]}
```

Dado um template, a geração dos pares questão-sparql pode ser realizada através da função ``build_pairs_from_template`` presente no [módulo generator](https://qapedia.readthedocs.io/pt/latest/QApedia.html#module-QApedia.generator) do **QApedia**.

a) Exemplo: Construindo pares de questão-sparql para o tipo de pergunta: $<$A$>$ foi um anime baseado em mangá?

In [3]:
template = {"question": "<A> foi um anime baseado em mangá?",
            "query": "ASK where {<A> dct:subject dbc:Anime_series_based_on_manga}",
            "generator_query": "select ?a where {?a dbo:type dbr:Manga . ?a dct:subject dbc:Anime_series_based_on_manga}",
            "variables": ["a"]}

pairs = generator.build_pairs_from_template(template)
len(pairs)

100

Foram geradas 100 pares de questão-sparql, abaixo é mostrada uma tabela contendo esses pares gerados.

In [4]:
df_pairs = pd.DataFrame.from_dict(pairs)
df_pairs

,question,sparql
0,Akame ga Kill! foi um anime baseado em mangá?,ASK where {<http://dbpedia.org/resource/Akame_ga_Kill!> dct:subject dbc:Anime_series_based_on_manga}
1,Nisekoi foi um anime baseado em mangá?,ASK where {<http://dbpedia.org/resource/Nisekoi> dct:subject dbc:Anime_series_based_on_manga}
2,Crayon Shin-chan foi um anime baseado em mangá?,ASK where {<http://dbpedia.org/resource/Crayon_Shin-chan> dct:subject dbc:Anime_series_based_on_manga}
3,Kuroko no Basket foi um anime baseado em mangá?,ASK where {<http://dbpedia.org/resource/Kuroko's_Basketball> dct:subject dbc:Anime_series_based_on_manga}
4,Kochira Katsushika-ku Kameari Kōen-mae Hashutsujo foi um anime baseado em mangá?,ASK where {<http://dbpedia.org/resource/Kochira_Katsushika-ku_Kameari_Kōen-mae_Hashutsujo> dct:subject dbc:Anime_series_based_on_manga}
5,Atashin'chi foi um anime baseado em mangá?,ASK where {<http://dbpedia.org/resource/Atashin'chi> dct:subject dbc:Anime_series_based_on_manga}
6,Akatsuki no Yona foi um anime baseado em mangá?,ASK where {<http://dbpedia.org/resource/Yona_of_the_Dawn> dct:subject dbc:Anime_series_based_on_manga}
7,Gakuen Alice foi um anime baseado em mangá?,ASK where {<http://dbpedia.org/resource/Gakuen_Alice> dct:subject dbc:Anime_series_based_on_manga}
8,A Channel foi um anime baseado em mangá?,ASK where {<http://dbpedia.org/resource/A_Channel_(manga)> dct:subject dbc:Anime_series_based_on_manga}
9,Seitokai Yakuindomo foi um anime baseado em mangá?,ASK where {<http://dbpedia.org/resource/Seitokai_Yakuindomo> dct:subject dbc:Anime_series_based_on_manga}


Os resultados retornados não estão utilizando as URIs de forma reduzida, para resolver isso basta passar os prefixos como argumento da função.

In [5]:
prefixes = "PREFIX dbr: <http://dbpedia.org/resource/>\
            PREFIX dbo: <http://dbpedia.org/ontology/>"
list_of_prefixes = utils.convert_prefixes_to_list(prefixes)
print(list_of_prefixes)
pairs = generator.build_pairs_from_template(template, prefixes, list_of_prefixes)

[('dbr:', 'http://dbpedia.org/resource/'), ('dbo:', 'http://dbpedia.org/ontology/')]


In [6]:
df_pairs = pd.DataFrame.from_dict(pairs)
df_pairs

,question,sparql
0,Medaka Box foi um anime baseado em mangá?,ASK where {dbr:Medaka_Box dct:subject dbc:Anime_series_based_on_manga}
1,Pet Shop of Horrors foi um anime baseado em mangá?,ASK where {dbr:Pet_Shop_of_Horrors dct:subject dbc:Anime_series_based_on_manga}
2,Himōto! Umaru-chan foi um anime baseado em mangá?,ASK where {dbr:Himouto!_Umaru-chan dct:subject dbc:Anime_series_based_on_manga}
3,Big Order foi um anime baseado em mangá?,ASK where {dbr:Big_Order dct:subject dbc:Anime_series_based_on_manga}
4,Saki (mangá) foi um anime baseado em mangá?,ASK where {dbr:Saki_(manga) dct:subject dbc:Anime_series_based_on_manga}
5,Ao no Exorcist foi um anime baseado em mangá?,ASK where {dbr:Blue_Exorcist dct:subject dbc:Anime_series_based_on_manga}
6,Gakuen Alice foi um anime baseado em mangá?,ASK where {dbr:Gakuen_Alice dct:subject dbc:Anime_series_based_on_manga}
7,Ai Shite Knight foi um anime baseado em mangá?,ASK where {dbr:Ai_Shite_Knight dct:subject dbc:Anime_series_based_on_manga}
8,Maison Ikkoku foi um anime baseado em mangá?,ASK where {dbr:Maison_Ikkoku dct:subject dbc:Anime_series_based_on_manga}
9,Mangaka-san to Assistant-san to foi um anime baseado em mangá?,ASK where {dbr:The_Comic_Artist_and_His_Assistants dct:subject dbc:Anime_series_based_on_manga}


## QApedia - Carregando arquivo de templates para geração

O arquivo dos templates deve se encontrar na seguinte estrutura.
```
question;query;generator_query
Qual tipo de governo de <A>?;SELECT ?uri WHERE{ <A> <http://pt.dbpedia.org/property/tipoGoverno> ?uri .};SELECT distinct ?a WHERE{ ?a <http://pt.dbpedia.org/property/tipoGoverno> ?uri . }
```

**Observações sobre o arquivo:**
* Cada linha representa um template
* Um template é definido no formato "question;query;generator_query"
* A primeira linha contém o cabeçalho
* As variavéis são extraídas automaticamente na leitura do arquivo, desde que a *generator_query* esteja em um formato válido.
    * As variáveis da *generator_query* são no formato $?a ?b .. ?y ?z$ e correspondem as lacunas $<A> <B> ... <Y> <Z>$
    * A *generator_query* não possui campo de label a fim de facilitar a sua reutilização para outros idiomas.
    
   
Para carregar o arquivo contendo os templates, use a função ``load_templates`` do módulo [io](https://qapedia.readthedocs.io/pt/latest/QApedia.html#module-QApedia.io). Caso esteja usando prefixos, o arquivo de prefixos pode ser carregado através da função ``load_prefixes`` que também se encontra disponível no módulo [io](https://qapedia.readthedocs.io/pt/latest/QApedia.html#module-QApedia.io).

In [7]:
prefixes, list_of_prefixes = io.load_prefixes("data/prefixes.txt")
templates = io.load_templates("data/example.csv")
print(f"Quantidade de prefixos: {len(list_of_prefixes)}")
print(f"Quantidade de templates: {len(templates)}")

Quantidade de prefixos: 9
Quantidade de templates: 5


**Arquivo contendo os templates**

In [8]:
templates

,question,query,generator_query,variables
0,<A> e <B> são produzidos por qual empresa?,SELECT DISTINCT ?uri where { <A> dbpedia2:manufacturer ?uri . <B> dbpedia2:manufacturer ?uri },select distinct ?a ?b where { ?a dbpedia2:manufacturer ?uri . ?b dbpedia2:manufacturer ?uri },"[a, b]"
1,<A> e <B> são os trabalhos notáveis de qual escritor(a)?,SELECT DISTINCT ?uri where { ?uri dbpedia2:notableworks <A> . ?uri dbpedia2:notableworks <B> . ?uri a dbo:writer },select distinct ?a ?b where { ?uri dbpedia2:notableworks ?a . ?uri dbpedia2:notableworks ?b . ?uri a dbo:writer },"[a, b]"
2,<A> e <B> são escritos por qual autor(a)?,SELECT DISTINCT ?uri where { <A> dbpedia2:writer ?uri . <B> dbo:author ?uri },select distinct ?a ?b where { ?a dbpedia2:writer ?uri . ?b dbo:author ?uri },"[a, b]"
3,<A> escreveu qual livro?,SELECT DISTINCT ?uri where { ?uri dbo:author <A> },select distinct ?a where { ?uri dbo:author ?a },[a]
4,<A> pertence a qual partido político?,SELECT DISTINCT ?uri where { <A> dbo:occupation ?uri },select distinct ?a where { ?a dbo:occupation ?uri },[a]


**Lista de prefixos definida**

1. prefixos sem pré-processamento

In [9]:
print(prefixes)

PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX : <http://dbpedia.org/resource/>
PREFIX dbpedia2: <http://dbpedia.org/property/>
PREFIX dbpedia: <http://dbpedia.org/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>


2. prefixos convertidos em lista

In [10]:
list_of_prefixes

[('owl:', 'http://www.w3.org/2002/07/owl#'),
 ('xsd:', 'http://www.w3.org/2001/XMLSchema#'),
 ('rdfs:', 'http://www.w3.org/2000/01/rdf-schema#'),
 ('rdf:', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#'),
 ('foaf:', 'http://xmlns.com/foaf/0.1/'),
 ('dc:', 'http://purl.org/dc/elements/1.1/'),
 ('dbpedia2:', 'http://dbpedia.org/property/'),
 ('dbpedia:', 'http://dbpedia.org/'),
 ('skos:', 'http://www.w3.org/2004/02/skos/core#')]

### Gerando pares de questão-sparql para cada template

No exemplo a seguir é gerado no máximo 10 pares por template, ou seja, haverá no máximo 50 pares de questão-sparql no total. Esses exemplos são salvos em um arquivo que foi definido como "pairs.csv". A geração dos pares abaixo levará alguns minutos dependendo da sua conexão com a Internet, já que a consulta é feita online utilizando o SPARQL endpoint da DBpedia.

In [11]:
with open("pairs.csv", "w") as csv_file:
    writer = csv.writer(csv_file, delimiter=";")
    writer.writerow(["question", "sparql", "template_id"])
    for index, template in templates.iterrows():
        print("Executando template da linha %d" % index)
        # Realizar a busca e construção dos pares questão-sparql
        pairs = generator.build_pairs_from_template(
            template,
            prefixes,
            list_of_prefixes,
            "http://dbpedia.org/sparql",
            number_of_examples=10,
        )
        for pair in pairs:
            writer.writerow([pair["question"], pair["sparql"], index])
    csv_file.close()

Executando template da linha 0
Executando template da linha 1
Executando template da linha 2
Executando template da linha 3
Executando template da linha 4


O arquivo contendo os pares de questão sparql pode ser carregado através do pandas.

In [12]:
pairs = pd.read_csv("pairs.csv", sep=";")
pairs

,question,sparql,template_id
0,Kosmos 634 e Kosmos 176 são produzidos por qual empresa?,SELECT DISTINCT ?uri where { dbpedia:Kosmos_634 dbpedia2:manufacturer ?uri . dbpedia:Kosmos_176 dbpedia2:manufacturer ?uri },0
1,Nahuel 1A e Tele-X são produzidos por qual empresa?,SELECT DISTINCT ?uri where { dbpedia:Nahuel_1A dbpedia2:manufacturer ?uri . dbpedia:Tele-X dbpedia2:manufacturer ?uri },0
2,Telstar 401 e JCSAT-110 são produzidos por qual empresa?,SELECT DISTINCT ?uri where { dbpedia:Telstar_401 dbpedia2:manufacturer ?uri . dbpedia:N-SAT-110 dbpedia2:manufacturer ?uri },0
3,Kosmos 165 e Kosmos 314 são produzidos por qual empresa?,SELECT DISTINCT ?uri where { dbpedia:Kosmos_165 dbpedia2:manufacturer ?uri . dbpedia:Kosmos_314 dbpedia2:manufacturer ?uri },0
4,Skynet 5A e Alphasat são produzidos por qual empresa?,SELECT DISTINCT ?uri where { dbpedia:Skynet_5A dbpedia2:manufacturer ?uri . dbpedia:Inmarsat-4A_F4 dbpedia2:manufacturer ?uri },0
5,DirecTV-14 e SES-5 são produzidos por qual empresa?,SELECT DISTINCT ?uri where { dbpedia:DirecTV-14 dbpedia2:manufacturer ?uri . dbpedia:SES-5 dbpedia2:manufacturer ?uri },0
6,Kosmos 76 e Kosmos 311 são produzidos por qual empresa?,SELECT DISTINCT ?uri where { dbpedia:Kosmos_76 dbpedia2:manufacturer ?uri . dbpedia:Kosmos_311 dbpedia2:manufacturer ?uri },0
7,Kosmos 580 e Kosmos 265 são produzidos por qual empresa?,SELECT DISTINCT ?uri where { dbpedia:Kosmos_580 dbpedia2:manufacturer ?uri . dbpedia:Kosmos_265 dbpedia2:manufacturer ?uri },0
8,Ariel 5 e COBE são produzidos por qual empresa?,SELECT DISTINCT ?uri where { dbpedia:Ariel_5 dbpedia2:manufacturer ?uri . dbpedia:Cosmic_Background_Explorer dbpedia2:manufacturer ?uri },0
9,Kosmos 173 e Kosmos 311 são produzidos por qual empresa?,SELECT DISTINCT ?uri where { dbpedia:Kosmos_173 dbpedia2:manufacturer ?uri . dbpedia:Kosmos_311 dbpedia2:manufacturer ?uri },0
